# Arbitrary State Generation

## Step 1: Computing pulse sequences

In this first section, we develop tools for computing the pulse sequences necessary for generating arbitrary resonator states. We follow the algorithm laid out in the paper, starting with the desired final state and then repetitively emptying the top Fock state into the qubit, and proceeding down the ladder to the ground state. 

First, let's import modules.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import *

Let's define some constants. 

In [2]:
t1q = 650  # qubit T1 in ns
t2q = 150  # qubit T2 in ns
t1r = 3500  # resonator T1 in ns
w_r = 2 * np.pi * 6.570  # resonator frequency in GHz
omega = 2 * np.pi * 19e-3  # qubit/cavity interaction strength in GHz
delta_on = 0  # qubit/cavity detuning when on resonance 
delta_off = - 2 * np.pi * 463e-3  # qubit/cavity detuning when off resonance in GHz

There are three types of pulses: swap operations, qubit drive operations, and phase rotation operations. Let's write some functions to calculate the length of time each step needs to be applied for. Let's start with the swap operation. 

When the qubit is tuned on resonance with the cavity, we observe Rabi oscillations between the state $| e, n-1\rangle$ and the state $|g, n\rangle$ at a frequency of $\sqrt{n}\Omega$. Thus, the appropriate length of time to leave the system tuned for is $\pi / \sqrt{n} \Omega$. So we have 

In [3]:
def swap_time(n):
    return np.pi / (omega * np.sqrt(n))

Next, we should figure out the qubit drive operations, as they 

Now let's think about the phase rotation operations. When the qubit is detuned from the resonator, the $|e, n\rangle$ states accumulate phase relative to the $|g, n-1\rangle$ states at a rate $\Delta_{off}$. We want these two states to be exactly in phase 